# 🔐 Divine Convergence – Full D1 Chart Parser with Firebase Upload

## Step 0: Upload Your Firebase Key

In [ ]:

from google.colab import files
uploaded = files.upload()

import json
import os

for fn in uploaded.keys():
    if fn.endswith('.json'):
        firebase_key_path = fn
        break


## Step 1: Install Required Packages

In [ ]:
!pip install firebase-admin

## Step 2: Initialize Firebase

In [ ]:

import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate(firebase_key_path)
firebase_admin.initialize_app(cred)
db = firestore.client()


## Step 3: Define Astro-Seek Parsing and Utilities

In [ ]:

import re
import urllib.parse

zodiac_signs = [
    "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
    "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
]

def degree_to_sign(degree_float):
    sign_index = int(degree_float // 30)
    degree_within_sign = round(degree_float % 30, 2)
    sign = zodiac_signs[sign_index]
    return f"{degree_within_sign:.2f}° {sign}"

def build_astroseek_chart_url(birth_data):
    base_url = "https://horoscopes.astro-seek.com/birth-chart-horoscope-online"
    params = {
        "narozeni_den": birth_data["day"],
        "narozeni_mesic": birth_data["month"],
        "narozeni_rok": birth_data["year"],
        "narozeni_hodina": birth_data["hour"],
        "narozeni_minuta": birth_data["minute"],
        "narozeni_mesto": birth_data["city"],
        "send": "Send"
    }
    query_string = urllib.parse.urlencode(params)
    return f"{base_url}?{query_string}"

def parse_astroseek_d1(url):
    pattern = re.compile(r"p_(sun|moon|mercury|venus|mars|jupiter|saturn|uran|neptun|pluto|uzel|lilith|chiron)=([\d.]+)")
    matches = pattern.findall(url.lower())

    chart_data = {}
    for planet, value in matches:
        degree = float(value)
        planet_name = planet.capitalize().replace("uzel", "North Node").replace("uran", "Uranus").replace("neptun", "Neptune").replace("pluto", "Pluto").replace("lilith", "Lilith").replace("chiron", "Chiron")
        chart_data[planet_name] = degree_to_sign(degree)

    return chart_data


## Step 4: Push Parsed Chart to Firebase

In [ ]:

def push_to_firebase(user_id, birth_data, chart_data):
    user_ref = db.collection('users').document(user_id)
    user_ref.set({
        'birthData': birth_data,
        'chartData': chart_data,
        'matchStatus': 'waiting'
    })
    print(f"Pushed {user_id} to Firebase.")


## Step 5: Example Usage

In [ ]:

birth_data = {
    "day": 2,
    "month": 7,
    "year": 1985,
    "hour": 5,
    "minute": 25,
    "city": "Baltimore"
}

astroseek_url = build_astroseek_chart_url(birth_data)
print("Generated Astro-Seek URL:", astroseek_url)


In [ ]:

# ⚠️ Manual Step: Paste chart PDF URL (with p_sun, etc.)
pdf_chart_url = "https://horoscopes.astro-seek.com/...&p_sun=100.43&p_moon=278.93&p_mercury=123.89..."

parsed_chart = parse_astroseek_d1(pdf_chart_url)
print("\nParsed D1 Chart:")
for planet, position in parsed_chart.items():
    print(f"{planet}: {position}")

push_to_firebase("user_test_19850702", birth_data, parsed_chart)
